# Chap 10 데이터 집계와 그룹 연산

 
* 하나 이상의 를 이용해서 pandas 객체를 여러 조각으로 나누는 법

* 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계 계산법

* 정규화, 선형회귀, 등급 또는 부분 집합 선택 등 집단 내 변형이나 다른 조작을 적용하는 방법

* 피벗테이블과 교차일람표를 구하는 방법

* 변위치 분석과 다른 통계 집단 분석을 수행하는 방법


## GroupBy 메카닉 


1. 분리 
 - 하나의 Key를 기준으로 **분리**
 
2. 적용
 - 함수를 나뉘어진 그룹에 **적용**시켜 새로운 값 얻어내기

3.  결합
 - 함수가 적용된 결과를 하나의 객체로 **결합**
 
 
---------------------------------

Google의 논문 [MapReduce: Simplified Data Processing on Large Clusters](https://research.google/pubs/pub62/)에 영향받은

데이터 분석에서는 Hadley Wickhan의 논문 [The Split-Apply-Combine Strategy for Data Analysis](https://www.jstatsoft.org/v40/i01/paper)

에서 시작되었음
 
------------------------------ 
> 두 논문 다 시간 날 때 읽어 보시길 권합니다..!

## 구글 맵 리듀스

![Mapreduce](Googles-MapReduce-programming-model.png)


[이미치 출처](https://www.researchgate.net/profile/Wubin-Pan/publication/262021460/figure/fig9/AS:669668532551696@1536672828942/Googles-MapReduce-programming-model.png)

## Split-Apply-Combine

![split-apply_combine](./01202014split-apply-combine.png)

[이미지 출처](https://andeekaplan.com/images/blog/01202014split-apply-combine.png)

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", "b", "b", "a"],
        "key2": ["one", "two", "one", "two", np.nan],
        "data1": np.random.randn(5),
        "data2": np.random.randn(5),
    }
)
df

,key1,key2,data1,data2
0,a,one,-1.960207,-0.140437
1,a,two,-1.336017,0.268550
2,b,one,1.002201,-1.373011
3,b,two,-0.790304,-0.408483
4,a,NaN,-1.812640,1.026374


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped  # 단순히 groupby만 쓰면 groupby 객체만 나옵니다.

In [4]:
grouped.mean()  # key1 칼럼 기준으로 묶어서 평균을 계산해주었음.
# 새로운 이 Series 객체의 색인은 우리가 groupby로 지정한 'key1임'

key1
a   -1.702955
b    0.105948
Name: data1, dtype: float64

In [5]:
means = (
    df["data1"].groupby([df["key1"], df["key2"]]).mean()
)  # groupby key를 key1, key2 두개를 줬기에 계층적 index를 갖게됨
means

key1  key2
a     one    -1.960207
      two    -1.336017
b     one     1.002201
      two    -0.790304
Name: data1, dtype: float64

In [6]:
means.unstack()  # unstack으로 풀어버리기도 가능

key2,one,two
key1,,
a,-1.960207,-1.336017
b,1.002201,-0.790304


In [7]:
states = np.array(["Ohio", "California", "California", "Ohio", "Ohio"])
years = np.array([2005, 2005, 2006, 2005, 2006])
df["data1"].groupby([states, years]).mean()  # 기존 data1에 없던 index를 넣어서도 가능함.

California  2005   -1.336017
            2006    1.002201
Ohio        2005   -1.375256
            2006   -1.812640
Name: data1, dtype: float64

In [8]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one  -1.960207 -0.140437
     two  -1.336017  0.268550
b    one   1.002201 -1.373011
     two  -0.790304 -0.408483

In [9]:
df.groupby(
    "key1"
).mean()  # 위와 비교하면 key2가 빠져있는데 key2는 숫자가 아니라 (범주형 데이터라) mean 계산이 불가능하기 때문
# 이런 칼럼을 성가신 칼럼 nuisance column이라 부르고 결과에서 제외함.

,data1,data2
key1,,
a,-1.702955,0.384829
b,0.105948,-0.890747


In [10]:
df.groupby(["key1", "key2"]).size()  # 이때 nan은 제외힘

key1  key2
a     one     1
      two     1
b     one     1
      two     1
dtype: int64

### 그룹간 순회하기

In [11]:
for name, group in df.groupby(
    "key1"
):  # 하나씩 for 문으로 받기도 가능. 뭔가 각 종류별로 데이터를 넣을 때 사용할 수 있을 듯 `
    print(name)
    print("-" * 10)
    print(group)
    print("\n")

a
----------
  key1 key2     data1     data2
0    a  one -1.960207 -0.140437
1    a  two -1.336017  0.268550
4    a  NaN -1.812640  1.026374


b
----------
  key1 key2     data1     data2
2    b  one  1.002201 -1.373011
3    b  two -0.790304 -0.408483




In [12]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print("-" * 10)
    print(group)
    print("\n")

('a', 'one')
----------
  key1 key2     data1     data2
0    a  one -1.960207 -0.140437


('a', 'two')
----------
  key1 key2     data1    data2
1    a  two -1.336017  0.26855


('b', 'one')
----------
  key1 key2     data1     data2
2    b  one  1.002201 -1.373011


('b', 'two')
----------
  key1 key2     data1     data2
3    b  two -0.790304 -0.408483




In [13]:
pieces = dict(list(df.groupby("key1")))  # dic 형식으로 받기 도 가능
pieces

{'a':   key1 key2     data1     data2
 0    a  one -1.960207 -0.140437
 1    a  two -1.336017  0.268550
 4    a  NaN -1.812640  1.026374,
 'b':   key1 key2     data1     data2
 2    b  one  1.002201 -1.373011
 3    b  two -0.790304 -0.408483}

In [14]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)  # 행을 기준이 아닌 열(axis = 1)을 기준으로도 그룹을 만들 수 있음
for dtype, group in grouped:
    print(dtype)
    print("-" * 20)
    print(group)
    print("\n")

float64
--------------------
      data1     data2
0 -1.960207 -0.140437
1 -1.336017  0.268550
2  1.002201 -1.373011
3 -0.790304 -0.408483
4 -1.812640  1.026374


object
--------------------
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  NaN




### 컬럼이나 컬럼의 일부만 선택하기 
생략

### dict과 Series에서 그룹핑하기

생략

### 함수로 그루핑하기 

In [16]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wes", "Jim", "Travis"],
)
people

,a,b,c,d,e
Joe,1.453443,0.391419,1.469604,1.944567,-0.241504
Steve,-0.224991,-0.019153,-0.331191,-0.300765,0.694537
Wes,-0.601667,-0.054579,-1.395512,-3.149635,0.199355
Jim,0.380179,-0.508040,-1.030185,-0.056848,0.074005
Travis,-0.377402,1.425381,0.329636,-0.287658,0.277808


In [17]:
people.groupby(
    len
).sum()  # axis = 0에 대해서 len()을 적용해서 3(Joe, Wes, Jim), 5(Steve),6(Travis) 순으로 정렬함

,a,b,c,d,e
3,1.231954,-0.171200,-0.956093,-1.261915,0.031856
5,-0.224991,-0.019153,-0.331191,-0.300765,0.694537
6,-0.377402,1.425381,0.329636,-0.287658,0.277808


In [18]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()  # len()의 결과와 key_list 값들 을 기준으로 multi index로 정렬함

a         b         c         d         e
3 one -0.601667 -0.054579 -1.395512 -3.149635 -0.241504
  two  0.380179 -0.508040 -1.030185 -0.056848  0.074005
5 one -0.224991 -0.019153 -0.331191 -0.300765  0.694537
6 two -0.377402  1.425381  0.329636 -0.287658  0.277808

### 인덱스 단계로 그루핑
생략

## 데이터 집계

자신만의 데이터 함수 사용하려면 agg메서드에 해당 함수를 넘기면 된다.

In [19]:
df
grouped = df.groupby("key1")

In [20]:
def peak_to_peak(arr):
    return arr.max() - arr.min()


grouped.agg(peak_to_peak)

C:\Users\WJ\anaconda3\envs\aiffel\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,0.624190,1.166811
b,1.792505,0.964529


### 컬럼에 여러 가지 함수 적용하기
생략

In [21]:
tips = pd.read_csv("./tips.csv")
# Add tip percentage of total bill
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [22]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [23]:
grouped = tips.groupby(["day", "smoker"])

In [24]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [25]:
grouped_pct.agg(["mean", peak_to_peak])

mean  peak_to_peak
day  smoker                        
Fri  No      0.151650      0.067349
     Yes     0.174783      0.159925
Sat  No      0.158048      0.235193
     Yes     0.147906      0.290095
Sun  No      0.160113      0.193226
     Yes     0.187250      0.644685
Thur No      0.160298      0.193350
     Yes     0.163863      0.151240

In [26]:
grouped_pct.agg([lambda x: len(x)])  # lambda 함수로만 표기가 되기에 알아보기 힘들기때문에

<lambda>
day  smoker          
Fri  No             4
     Yes           15
Sat  No            45
     Yes           42
Sun  No            57
     Yes           19
Thur No            45
     Yes           17

In [27]:
grouped_pct.agg([("count와 같은 효과", lambda x: len(x))])  # 표기될 함수명, 함수

count와 같은 효과
day  smoker              
Fri  No                 4
     Yes               15
Sat  No                45
     Yes               42
Sun  No                57
     Yes               19
Thur No                45
     Yes               17

In [28]:
grouped_pct.agg(
    [("평균", "mean"), ("최대값 - 최솟값", peak_to_peak)]
)  # 함수명이 다소 복잡하다면 (표기될 함수명, 함수) 도 가능

평균  최대값 - 최솟값
day  smoker                     
Fri  No      0.151650   0.067349
     Yes     0.174783   0.159925
Sat  No      0.158048   0.235193
     Yes     0.147906   0.290095
Sun  No      0.160113   0.193226
     Yes     0.187250   0.644685
Thur No      0.160298   0.193350
     Yes     0.163863   0.151240

In [29]:
ftuples = [("평균", "mean"), ("편차", np.var)]
grouped["tip_pct", "total_bill"].agg(ftuples)  # 튜플의 리스트로도 넘길 수 있음

C:\Users\WJ\anaconda3\envs\aiffel\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


tip_pct           total_bill            
                   평균        편차         평균          편차
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [30]:
grouped.agg({"tip": np.max, "size": "sum"})  # dict 형태로 넣어준다면 컬럼마다 다른 함수도 적용 가능
grouped.agg({"tip_pct": ["min", "max", "mean", "std"], "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 색인없이 집계된 데이터 받기

In [31]:
tips.groupby(["day", "smoker"], as_index=False).mean()  # 혹은 reset_index 메서드로 받아버리기도 가능.

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: 일반적인 분리-적용-병합

In [32]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(by=column)[-n:]


top(tips, n=6)  # 단순히 적용

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [33]:
tips.groupby("smoker").apply(top)  # 그룹에 분리해서 적용된후-다시 병합된것

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [34]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [35]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


### 그룹 Index 생략한 결과 받기

In [36]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 변위치(quntile) 분석과 버킷 분석

In [39]:
frame = pd.DataFrame({"data1": np.random.randn(1000), "data2": np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles  # frame.data1.을  4간격으로 나눈 [(-3.725, -1.94] < (-1.94, -0.163] < (-0.163, 1.615] < (1.615, 3.392]] 안에 속하게함

0      (-1.94, -0.163]
1      (-0.163, 1.615]
2      (-0.163, 1.615]
3      (-0.163, 1.615]
4      (-0.163, 1.615]
            ...       
995    (-0.163, 1.615]
996    (-0.163, 1.615]
997    (-1.94, -0.163]
998    (-0.163, 1.615]
999    (-1.94, -0.163]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64, right]): [(-3.725, -1.94] < (-1.94, -0.163] < (-0.163, 1.615] < (1.615, 3.392]]

위에서 만든 cut으로 만들어진 Category 객체는 groupby로 넘기기 가능.

In [40]:
def get_stats(group):
    return {
        "min": group.min(),
        "max": group.max(),
        "count": group.count(),
        "mean": group.mean(),
    }


grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.725, -1.94]",-2.342286,3.401496,22.0,0.132688
"(-1.94, -0.163]",-3.216935,2.969581,393.0,0.047044
"(-0.163, 1.615]",-2.933719,3.192032,525.0,-0.038073
"(1.615, 3.392]",-2.395201,1.885132,60.0,-0.038465


In [43]:
# Return quantile numbers. 이제 크기가 같은 버킷으로 계산 -> count가 전부 100
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats)

data1       
0      min       -2.342286
       max        3.401496
       count    100.000000
       mean       0.155853
1      min       -2.977708
       max        2.665962
       count    100.000000
       mean      -0.015356
2      min       -3.216935
       max        2.295222
       count    100.000000
       mean       0.070779
3      min       -2.964409
       max        2.969581
       count    100.000000
       mean       0.024511
4      min       -2.933719
       max        2.382050
       count    100.000000
       mean      -0.153570
5      min       -2.867240
       max        2.717802
       count    100.000000
       mean      -0.130254
6      min       -2.465333
       max        1.884206
       count    100.000000
       mean       0.061754
7      min       -2.111046
       max        2.021835
       count    100.000000
       mean       0.071092
8      min       -2.430384
       max        3.192032
       count    100.000000
       mean       0.012958
9      min     

In [48]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.342286,3.401496,100.0,0.155853
1,-2.977708,2.665962,100.0,-0.015356
2,-3.216935,2.295222,100.0,0.070779
3,-2.964409,2.969581,100.0,0.024511
4,-2.933719,2.382050,100.0,-0.153570
5,-2.867240,2.717802,100.0,-0.130254
6,-2.465333,1.884206,100.0,0.061754
7,-2.111046,2.021835,100.0,0.071092
8,-2.430384,3.192032,100.0,0.012958


### 예제1: 그룹마다 다른 값으로 결측치 채우기 

기본. 전체 평균으로 결측치 채우기

In [50]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0    0.195821
1    0.667090
2    0.195821
3   -1.163950
4    0.195821
5    1.084323
dtype: float64

응용. 그룹마다 다른 값으로 채우기

In [53]:
states = [
    "Ohio",
    "New York",
    "Vermont",
    "Florida",
    "Oregon",
    "Nevada",
    "California",
    "Idaho",
]
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.491101
New York      0.779267
Vermont       1.034824
Florida       0.271007
Oregon       -0.560582
Nevada       -0.923753
California    0.617793
Idaho        -1.387574
dtype: float64

In [54]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.491101
New York      0.779267
Vermont            NaN
Florida       0.271007
Oregon       -0.560582
Nevada             NaN
California    0.617793
Idaho              NaN
dtype: float64

In [55]:
group_key = ["East"] * 4 + ["West"] * 4
data.groupby(group_key).mean()  # 동부, 서부 별 평균 구하기

East    0.186391
West    0.028605
dtype: float64

In [56]:
fill_mean = lambda g: g.fillna(g.mean())  # 채워넘는 함수 정의
data.groupby(group_key).apply(fill_mean)

Ohio         -0.491101
New York      0.779267
Vermont       0.186391
Florida       0.271007
Oregon       -0.560582
Nevada        0.028605
California    0.617793
Idaho         0.028605
dtype: float64

In [57]:
fill_values = {"East": 0.5, "West": -1}
fill_func = lambda g: g.fillna(
    fill_values[g.name]
)  # 각 group은 내부적으로 name 속성('East', 'West')을 가지기에 그것을 활용함.
data.groupby(group_key).apply(fill_func)

Ohio         -0.491101
New York      0.779267
Vermont       0.500000
Florida       0.271007
Oregon       -0.560582
Nevada       -1.000000
California    0.617793
Idaho        -1.000000
dtype: float64

### 예제2: 랜덤 표본과 순열

실무에서는 워낙 데이터가 많으면 표본을 뽑아서 분석을 진행하기도 한다.

In [59]:
# Hearts, Spades, Clubs, Diamonds
suits = ["H", "S", "C", "D"]
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in ["H", "S", "C", "D"]:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [60]:
def draw(deck, n=5):
    return deck.sample(n)


draw(deck)

4H      4
4S      4
5S      5
9C      9
10D    10
dtype: int64

In [61]:
get_suit = lambda card: card[-1]  # last letter is suit  마지막 알파벳으로 선택됨
deck.groupby(get_suit).apply(draw, n=2)  # 마지막 알파벳으로 그룹 -> drav 2장식 선택

C  7C     7
   5C     5
D  3D     3
   9D     9
H  KH    10
   8H     8
S  4S     4
   2S     2
dtype: int64

In [62]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

AC      1
KC     10
KD     10
3D      3
5H      5
10H    10
6S      6
5S      5
dtype: int64

### 예제3: 그룹 별 가중평균과 상관관계

In [63]:
df = pd.DataFrame(
    {
        "category": ["a", "a", "a", "a", "b", "b", "b", "b"],
        "data": np.random.randn(8),
        "weights": np.random.rand(8),
    }
)
df

,category,data,weights
0,a,0.297976,0.611788
1,a,0.235856,0.005977
2,a,-3.485964,0.496860
3,a,-0.107553,0.594763
4,b,1.562997,0.583950
5,b,0.016775,0.665758
6,b,-0.236675,0.957144
7,b,-0.588213,0.517196


In [64]:
grouped = df.groupby("category")
get_wavg = lambda g: np.average(g["data"], weights=g["weights"])
grouped.apply(get_wavg)  ## category a , b 별로 계싼하되 weight를 둬서 계산

category
a   -0.943201
b    0.144317
dtype: float64

야후 주식데이터 사용

In [66]:
close_px = pd.read_csv("./stock_px_2.csv", parse_dates=True, index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [67]:
spx_corr = lambda x: x.corrwith(x["SPX"])  # S&P는 상위 500개 시총만 모아놓은것

In [71]:
rets = close_px.pct_change().dropna()  # 일일 퍼센트 변화율을 계산후 dropna() (무조건 하루는 na가 생기기 때문)
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [69]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)  # year로 묶은후 SPX와 상관관계 계산

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [70]:
by_year.apply(lambda g: g["AAPL"].corr(g["MSFT"]))  # 단순 칼럼끼리도 상관관계 게산 가능

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 예제4: 그룹 안에서의 linear regression(선형 회귀)

In [74]:
import statsmodels.api as sm


def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.0
    result = sm.OLS(Y, X).fit()
    return result.params

In [76]:
by_year.apply(regress, "AAPL", ["SPX"])  # 애플 주식에 대한 SPX의 연간 선형 회귀

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 피벗 테이블과 교차일람표(Cross_Tabulation)

데이터를 하나 이상의 키로 수집해서 어떤 키는 행에 어떤 키는 열에 나열해서 데이터를 정렬함.

groupby를 활용하기 위해서 margin(마진)이라는 부분합 추가 가능.

df의 메서드지만 pivot_table은 pandas 모듈의 최상위 함수로도 존재.

In [78]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [77]:
tips.pivot_table(index=["day", "smoker"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [79]:
tips.pivot_table(["tip_pct", "size"], index=["time", "day"], columns="smoker")

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [81]:
tips.pivot_table(
    ["tip_pct", "size"],
    index=["time", "day"],  # all 행, 열 추가로 그룹 통계 얻을 수 있음
    columns="smoker",
    margins=True,
)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [82]:
tips.pivot_table(
    "tip_pct", index=["time", "smoker"], columns="day", aggfunc=len, margins=True
)  # 다른 집게함수는 aggfunc로 넘기면됨. len이나 count는 빈도를 반환

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [83]:
tips.pivot_table(
    "tip_pct",
    index=["time", "size", "smoker"],
    columns="day",
    aggfunc="mean",
    fill_value=0,
)  # fill_value로 결측치 채우기도 가능

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 교차 일람표(Cross-Tabulations: Crosstab)

In [87]:
from io import StringIO

data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [89]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [90]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## 결론

Groupby는 매우 유용하다.